In [ ]:
import webif as quickdb

### Count all objects

In [ ]:
sql = '''
SELECT
    count(*)
FROM
    pdr1_wide
'''

%time quickdb.dataframe(sql)

### Count clean objects

In [ ]:
filters = 'grizy'
flags = '''
    centroid_sdss_flags flags_pixel_edge
    flags_pixel_interpolated_center flags_pixel_saturated_center
    flags_pixel_cr_center flags_pixel_bad cmodel_flux_flags
    '''.split()

sql = f'''
SELECT
    count(*)
FROM
    pdr1_udeep
WHERE
    ref."detect_is-primary"
    AND { ' AND '.join(f'forced.{f}.id NOTNULL' for f in filters) }
    AND { ' AND '.join(f'NOT forced.{f}.{flag}' for f in filters for flag in flags) }
'''

%time quickdb.dataframe(sql)

### Count primary objects

In [ ]:
sql = '''
SELECT
    count(*)
FROM
    pdr1_wide
GROUP BY
    ref.is_primary
'''
%time quickdb.dataframe(sql)

### Magnitude Histogram

In [ ]:
sql = '''
    SELECT
        histogram(f2m(forced.i.flux_sinc), bins => 200) as hist,
        count(*)
    FROM
        pdr1_wide
    WHERE
        ref."detect_is-primary"
    GROUP BY
        forced.i.classification_extendedness < 0.5
'''

%time hist1 = quickdb.dataframe(sql)
hist1

In [ ]:
%matplotlib inline
from matplotlib import pyplot

for i, row in hist1.iterrows():
    hist, bins = row['hist']
    group = row['group']
    pyplot.plot(bins[1:], hist, label=f'group: {group}')
pyplot.grid()
pyplot.legend()

## 2D Histogram

### Stellar sequence

In [ ]:
sql = f'''
SELECT
    histogram2d(
        f2m(forced.g.flux_sinc) - f2m(forced.r.flux_sinc),    
        f2m(forced.r.flux_sinc) - f2m(forced.i.flux_sinc),
        bins => (200, 400),
        range => ((0, 1.5), (-0.5, 2.5))
    )
FROM
    pdr1_wide
WHERE
    ref.classification_extendedness < 0.5
'''

%time stellarsequence = quickdb.dataframe(sql)
display(stellarsequence)

In [ ]:
import numpy

hist, xedges, yedges = stellarsequence['c0'][0]
pyplot.imshow(numpy.log(1 + hist.T), origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

### Spatial number count

In [ ]:
sql = f'''
SELECT
    histogram2d(
        ref.coord[0] * degree,
        ref.coord[1] * degree,
        bins => (200, 200)
    ) as count

FROM pdr1_wide
WHERE
    ref.coord[0] * degree BETWEEN 210 AND 240 AND
    ref.coord[1] * degree BETWEEN -10 AND 10 AND
    ref."detect_is-primary"
    '''

%time numbercount = quickdb.dataframe(sql)
hist, xedges, yedges = numbercount['count'][0]

pyplot.imshow(hist.T, origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

In [ ]:
sql = f'''
SELECT
    histogram2d(
        ref.coord[0] * degree,
        ref.coord[1] * degree,
        bins => (200, 200)
    )
FROM
    pdr1_udeep
WHERE
    ref."detect_is-primary"
GROUP BY
    ref.coord[0] * degree // 30,
    ref.coord[1] * degree // 30
'''

# %time numbercount2 = quickdb.dataframe(sql)

# for hist, xedges, yedges in numbercount2['c0']:
#     pyplot.imshow(numpy.log(hist.T+1), origin='low', extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
#     pyplot.show()

### Crossmatch

#### install hscmap jupyterLab extension

https://hsc-gitlab.mtk.nao.ac.jp/ssp-software/jupyterlab-hscmap

Open hscmap window

In [ ]:
import hscmap
w = hscmap.Window()

#### Generate random coordinates for testing

In [ ]:
import numpy

def gen_coord(n):
    a0 = 331
    a1 = 340
    d0 = -0.58
    d1 = 2
    r = numpy.random.uniform(a0, a1, n)
    d = numpy.random.uniform(d0, d1, n)
    return numpy.array([r, d])

my_cat = gen_coord(10000)

### Plot randomly generate points

In [ ]:
w.jump_to(335, 0, 5)
w.catalogs.clear()
w.catalogs.new(*my_cat, color=[0, 1, 0, 0.25])

In [ ]:
sql = f'''
SELECT
    crossmatch(
        ref.coord,                -- reference catalog
        shared.my_cat / degree,   -- user's catalog [radian]
        5 / arcsec,               -- match radius   [radian]
        ref.id,                   -- columns to extract[0]
        ref.coord * degree,       --                   [1]
        f2m(forced.i.flux_sinc)   --                   [2]
    ) as crossmatch
FROM
    pdr1_wide
WHERE
    ref."detect_is-primary"
    -- AND forced.i.classification_extendedness < 0.5
'''

%time df = quickdb.dataframe(sql, shared={'my_cat': my_cat.T})
df

#### Plot matched pairs

In [ ]:
cm = df['crossmatch'][0]

my_cat_index, extracted_columns = cm
matched_coord = extracted_columns[1]

w.catalogs.clear()
m1 = w.catalogs.new(*my_cat)
m2 = w.catalogs.new(*matched_coord.T, color=[1, 0, 0, 0.5])


In [ ]:
extracted_columns[2] # f2m(forced.i.flux_sinc)